In [ ]:
import pandapower.plotting as plot
import xlsxwriter as xw
import csv
from math import ceil
import os

import pandapower as pp
from pandapower.networks import case1354pegase
%config InlineBackend.figure_format = 'png'
plt.rcParams['figure.figsize'] = (4, 3)
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'Times New Roman'

In [3]:
# Get bus
all_bus = np.arange(0, net.bus.shape[0])

# Manually set renewable generator bus 
wind_bus = pd.read_csv('zones/wind_bus.csv', index_col=None, header=None).to_numpy().reshape((-1))
wind_bus = wind_bus - 1 # Bus should be 0-indexed
wind_bidx = np.isin(net.gen.bus.values, wind_bus) # Get the Boolean index of renewable generators

# Get generator bus bidx (slack bus NOT included)
gen_bidx = np.isin(all_bus, net.gen.bus.values)
# Get generator bus no (including renewable, thermal and slack buses)
# slack_gen_no = np.isin(net.bus.name-1, net.gen.bus)

# Get load buses no
load_bidx = np.isin(all_bus, net.load.bus.values)

NameError: name 'net' is not defined

In [ ]:
# The array to store 'p_mw' of net.res_bus
bus_features = ['p_mw']
p_bus_list = np.empty((net.bus.shape[0], len(bus_features)))

# The arrry to store corresponding results 'net.res_gen.p_mw' & 'net.res_gen.va_degree'
# For the time being, we only consider 'p_mw'
# p_gen_features = ['p_mw', 'va_degree']
gen_features = ['p_mw']
p_gen_list = np.empty((net.gen.shape[0]*len(gen_features), 1))
p_max_gen_list = np.empty((net.gen.shape[0], 1))

# The array to store p_mw at load buses
p_load_list = np.empty((net.load.shape[0], 1))

# The array to store corresponding results 'net.res.line...' (there are multiple features in line result).
# Only need to keep the following five features in DC OPF.
# For the time being, we only care about 'p_mw'
# pf_line_features = ['p_from_mw', 'i_ka', 'va_from_degree', 'va_to_degree', 'loading_percent']
line_features = ['p_from_mw']
pf_line_list = np.empty((net.line.shape[0]*len(line_features), 1))
lp_line_list = np.empty((net.line.shape[0], 1))

# The array to store slack bus 'max_p_mw'
p_slack_max_list = []

# Global features
shedding_reserve_list = []
total_cost_list = []

In [ ]:
# Set parameters with significantly larger values for load data
num_rv = 1000  # Number of random variables
# Set distribution parameters
mu = np.array([50, 75, 100, 48, 81, 98, 52, 83, 45, 85, 98, 53, 81, 98, 47, 85])
std = np.array([15, 20, 15, 12, 21, 15, 13, 20, 12, 18, 16, 13, 20, 15, 12, 20])
rv_min = np.array([10, 25, 60, 9, 23, 60, 7, 21, 14, 25, 55, 9, 23, 55, 8, 21])
rv_max = np.array([90, 125, 140, 87, 139, 136, 97, 145, 76, 145, 141, 97, 139, 141, 86, 149])

# Generate distributions and random variables (RVs)
for i in range(1, num_zones + 1):
    # Left / right truncation in terms of standard deviation
    a = (rv_min[i - 1] - mu[i - 1]) / std[i - 1]
    b = (rv_max[i - 1] - mu[i - 1]) / std[i - 1]

    var_name = f'lx{i}'
    globals()[var_name] = np.linspace(rv_min[i - 1], rv_max[i - 1], num_rv)

    # Generate distributions
    var_name = f'tn{i}'
    globals()[var_name] = stats.truncnorm(loc=mu[i - 1], scale=std[i - 1], a=a, b=b)

    # Generate random variables and corresponding PDF
    var_name = f'tn_rv{i}'
    globals()[var_name] = globals()[f'tn{i}'].rvs(size=num_rv)
    var_name = f'tn_pdf{i}'
    globals()[var_name] = globals()[f'tn{i}'].pdf(x=globals()[f'lx{i}'])

# Plot load RVs
fig = plt.figure(figsize=(10, 4))

for i in range(1, num_zones + 1):
    ax = fig.add_subplot(4, 4, i)
    ax.plot(globals()[f'lx{i}'], globals()[f'tn_pdf{i}'])
    _, _, _ = ax.hist(globals()[f'tn_rv{i}'], bins=50, density=True, alpha=0.5)
    ax.set_xlim(1000, 14500)

plt.show()
plt.close('all')

In [ ]:


def windSpeedToPower(w):
    """
    input:
        w: wind speed (m/s), ndarray
    output:
        P: wind power (MW), ndarray
    """
    # Check the input type
    if type(w) != np.ndarray:
        raise TypeError("'w' should be 'numpy.ndarray'.")
    
    # Set parameters for wind power generation
    Pr = 100.  # Max wind power generation capacity (in MW)
    wi = 1.    # Wind speed below which there is no power generation (in m/s)
    wr = 15.   # Wind speed at which max power is generated (in m/s)
    
    # Wind power calculation
    P = np.zeros(w.shape)
    
    # Calculate power for wind speeds between cut-in and rated wind speed
    mask1 = (w >= wi) & (w < wr)
    P[mask1] = Pr * ((w[mask1] - wi) / (wr - wi)) ** 3
    
    # Assign maximum power for wind speeds above rated wind speed
    P[w >= wr] = Pr
    
    return P

In [ ]:
w = np.linspace(0, 17, 100)
P = windSpeedToPower(w)
plt.plot(w, P)
plt.vlines(x=0.5, ymin=0, ymax = 100, color='red', linestyles='dashed')
plt.vlines(x=15., ymin=0, ymax = 100, color='red', linestyles='dashed')
plt.xlabel('Wind speed ($v$)')
plt.ylabel('Wind power ($P$)')
plt.show()
plt.close('all')

In [ ]:

# Set parameters
loc = 0.
scale = np.array([10.0, 10.2, 14.0, 10.1, 9.9, 9.7, 10.0, 10.3, 9.8, 10.1, 14.5, 10.5, 9.5, 9.7, 10.0, 10.3])
shape = np.array([4.0, 3.8, 10.0, 3.9, 4.1, 3.9, 4.1, 4.0, 3.8, 3.8, 10.1, 4.2, 3.8, 4.0, 4.0, 4.5])


# Generate distributions and RVs
lx = np.linspace(0, 30, num_rv)
for i in range(1, num_zones+1):
    # Generate distributions
    var_name = f'twb{i}'
    globals()[var_name] = stats.weibull_min(loc=loc, c=shape[i-1], scale=scale[i-1])

    # Generate random variables and corresponding PDF
    var_name = f'twb_rv{i}'
    globals()[var_name] = globals()[f'twb{i}'].rvs(size=num_rv)
    var_name = f'twb_pdf{i}'
    globals()[var_name] = globals()[f'twb{i}'].pdf(x=lx)

    # Conver wind speed to wind power
    var_name = f'wind_power{i}'
    globals()[var_name] = windSpeedToPower(globals()[f'twb_rv{i}'])

In [ ]:
num_zones = 8  # Number of zones
os.makedirs('zones', exist_ok=True)  # Ensure the 'zones' directory exists

In [ ]:
## Read correlation matrix
corr_matrix = pd.read_csv('misc/corr_mat.csv', index_col=None, header=None).to_numpy()

## Read load /wind proportions
for i in range(1, num_zones+1):
    var_name = f'zone{i}_load_proportion'
    globals()[var_name] = pd.read_csv(f'zones/zone{i}_load_proportion.csv', 
                                      index_col=None, header=None).to_numpy().reshape((-1))
    
    var_name = f'zone{i}_wind_proportion'   
    globals()[var_name] = pd.read_csv(f'zones/zone{i}_wind_proportion.csv', 
                                      index_col=None, header=None).to_numpy().reshape((-1))

In [ ]:

# Load gen/load bus in individual zone (globals() dynamicly create variables)
num_zones = 8
for i in range(1, num_zones+1):
    # Load bus
    var_name = f'zone{i}_load_bus'
    globals()[var_name] = pd.read_csv(f'zones/{var_name}.csv', index_col=None, header=None).to_numpy() 
    var_name = f'zone{i}_load_bidx'
    globals()[var_name] = np.isin(net.load.bus, globals()[f'zone{i}_load_bus'])

    # Gen bus
    var_name = f'zone{i}_gen_bus'
    globals()[var_name] = pd.read_csv(f'zones/{var_name}.csv', index_col=None, header=None).to_numpy() 
    var_name = f'zone{i}_gen_bidx'
    globals()[var_name] = np.isin(net.gen.bus, globals()[f'zone{i}_gen_bus'])

    # Wind bus
    var_name = f'zone{i}_wind_bus'
    globals()[var_name] = pd.read_csv(f'zones/{var_name}.csv', index_col=None, header=None).to_numpy() 
    var_name = f'zone{i}_wind_bidx'
    globals()[var_name] = np.isin(net.gen.bus, globals()[f'zone{i}_wind_bus'])

In [ ]:
## Plot wind speed RVs
fig = plt.figure(figsize=(10, 4))

for i in range(1, num_zones+1):
    ax = fig.add_subplot(4, 4, i)
    ax.plot(lx, globals()[f'twb_pdf{i}'])
    _, _, _ = ax.hist(globals()[f'twb_rv{i}'], bins=50, density=True, alpha=0.5)
    ax.set_xlim(0, 30)

plt.show()
plt.close('all')

In [ ]:
## Plot wind power
fig = plt.figure(figsize=(10, 4))

for i in range(1, num_zones+1):
    ax = fig.add_subplot(4, 4, i)
    _, _, _ = ax.hist(globals()[f'wind_power{i}'], bins=50, density=True, alpha=0.5)
    ax.set_xlim(0, 200)

plt.show()
plt.close('all')

In [ ]:
## Number of grid variable samples
max_iteration = 100

## Numbe rof target OPF samples (solutions)
num_samples = 100

## Generate correlated RV
# Generate multivariate normal PDF (Standard normal PDF --> covariance = correlation)
mvnorm = stats.multivariate_normal(cov=corr_matrix)
mvnorm_rv = mvnorm.rvs(size=max_iteration)
# Get corresponding CDF (CANNOT USE MULTIVARIATE_NOMRAL CDF)
rv_cdf = stats.norm.cdf(mvnorm_rv)
# Generate RV under target PDFs using inverse CDF
for i in range(1, num_zones+1):
    var_name = f'L{i}'
    globals()[var_name] = globals()[f'tn{i}'].ppf(rv_cdf[:, i-1])
    var_name = f'R{i}'
    globals()[var_name] = windSpeedToPower(globals()[f'twb{i}'].ppf(rv_cdf[:, i+num_zones//2]))

for i in range(1, num_zones+1):
    var_name = f'L{i}'
    globals()[var_name] = globals()[f'tn{i}'].ppf(rv_cdf[:, i-1])
    var_name = f'R{i}'
    globals()[var_name] = windSpeedToPower(globals()[f'twb{i}'].ppf(rv_cdf[:, i-1]))

In [ ]:
j = 1
for i in range(1, num_zones+1):
    # Generate values for the rest of loads and RES
    var_name = f'L_{i}'
    globals()[var_name] = globals()[f'L{i}'][j] * globals()[f'zone{i}_load_proportion']
    var_name = f'R_{i}'
    globals()[var_name] = globals()[f'R{i}'][j] * globals()[f'zone{i}_wind_proportion']

    # # Set p_mw at load buses
    net.load.p_mw[globals()[f'zone{i}_load_bidx']] = globals()[f'L_{i}']

    # # Set max_p_mw adn min_p_mw at wind buses
    net.gen.max_p_mw[globals()[f'zone{i}_wind_bidx']] = globals()[f'R_{i}']
    net.gen.min_p_mw[globals()[f'zone{i}_wind_bidx']] = globals()[f'R_{i}']

    print(i)

In [ ]:
# Set RV index
j = 0   # Index for max_iteration
num = 0 # Index for OPF solutions
while j < max_iteration:
    ## Set p_mw at load buses and max_p_mw / min_p_mw at wind buses
    for i in range(1, num_zones+1):
        # Generate values for the rest of loads and RES
        var_name = f'L_{i}'
        globals()[var_name] = globals()[f'L{i}'][j] * globals()[f'zone{i}_load_proportion']
        var_name = f'R_{i}'
        globals()[var_name] = globals()[f'R{i}'][j] * globals()[f'zone{i}_wind_proportion']

        # Set p_mw at load buses
        net.load.p_mw[globals()[f'zone{i}_load_bidx']] = globals()[f'L_{i}']

        # Set max_p_mw adn min_p_mw at wind buses
        net.gen.max_p_mw[globals()[f'zone{i}_wind_bidx']] = globals()[f'R_{i}']
        net.gen.min_p_mw[globals()[f'zone{i}_wind_bidx']] = globals()[f'R_{i}']
    
    # Set 'max_p_mw' at slack bus 
    if net.load.p_mw.sum() >= net.gen.max_p_mw.sum():
        net.ext_grid.max_p_mw = net.load.p_mw.sum() - net.gen.max_p_mw.sum()
    else:
        net.ext_grid.max_p_mw = 0.

    ## Run OPF solver (ignore unconverged cases) and collect data
    try:
        ## Run DCOPP
        pp.rundcopp(net)

        ## Collect data
        # p_mw at load buses
        p_load = net.load.p_mw.to_numpy().reshape((-1, 1))
        p_load_list = np.concatenate([p_load_list, p_load], axis=1)
        # max_p_mw at gen buses
        p_max_gen = net.gen.max_p_mw.to_numpy().reshape((-1, 1))
        p_max_gen_list = np.concatenate([p_max_gen_list, p_max_gen], axis=1)
        # max_p_mw at slack bus
        p_slack_max_list.append(net.ext_grid.max_p_mw.values)

        ## Collect results
        # Results of p_mw at all buses
        # order='F' means first index changes fastest while last index changes slowest
        p_bus = net.res_bus[bus_features].to_numpy().reshape((-1, len(bus_features)), order='F') 
        p_bus_list = np.concatenate([p_bus_list, p_bus], axis=1)
        # Results of p_mw at gen buses
        p_gen = net.res_gen[gen_features].to_numpy().reshape((-1, 1), order='F')  
        p_gen_list = np.concatenate([p_gen_list, p_gen], axis=1)
        # Results of power flow at all lines
        pf_line = net.res_line[line_features].to_numpy().reshape((-1, 1), order='F')
        pf_line_list = np.concatenate([pf_line_list, pf_line], axis=1)
        # Results of loading percent at all lines
        lp_line = net.res_line.loading_percent.to_numpy().reshape((-1, 1))
        lp_line_list = np.concatenate([lp_line_list, lp_line], axis=1)
        # Results of shedding and reserve
        shedding_reserve = net.load.p_mw.sum() - net.gen.max_p_mw.sum()
        shedding_reserve_list.append(shedding_reserve)
        # Results of total cost
        total_cost = net.res_cost / 1e3
        total_cost_list.append(total_cost)

        # Increment index
        num += 1

        if num >= num_samples:
            print(f'{num_samples} OPF solutions found')
            break
    except:
        print(f'No solution found for sample {j+1}')
        pass

    # Increment index
    j += 1

    # Increment index
    if j >= max_iteration:
        print(f'No sufficient samples found for {num_samples} OPF solutions')
        break

In [ ]:
# Save synthetic data (node level)
pd.DataFrame(p_load_list[:, 1:]).to_csv('synthetic_data/p_load_data.csv')
pd.DataFrame(p_max_gen_list[:, 1:]).to_csv('synthetic_data/p_max_gen_data.csv')
pd.DataFrame(p_slack_max_list).to_csv('synthetic_data/p_slack_max_data.csv')
pd.DataFrame(p_bus_list[:, len(bus_features):]).to_csv('synthetic_data/p_bus_data.csv')
pd.DataFrame(p_gen_list[:, 1:]).to_csv('synthetic_data/p_gen_data.csv')

# Save synthetic data (edge level)
pd.DataFrame(pf_line_list[:, 1:]).to_csv('synthetic_data/pf_line_data.csv')
pd.DataFrame(lp_line_list[:, 1:]).to_csv('synthetic_data/lp_line_data.csv')

pd.DataFrame(lp_line_list[:, 1:]).to_csv('reliability_risk/lp_true_corr.csv')

# Save synthetic data (graph level)
graph_features = pd.concat([pd.Series(shedding_reserve_list, name='shedding_reserve'), 
                           pd.Series(total_cost_list, name='total_cost(10^3)')], 
                           axis=1)
graph_features.T.to_csv('synthetic_data/graph_features.csv')

In [ ]:
lp_line = pd.DataFrame(lp_line_list[:, 1:])

In [ ]:
_, _, _ = plt.hist(lp_line.values.flatten(), bins=50, alpha=0.5)
plt.xscale('log')
plt.yscale('log')
plt.show()
plt.close('all')

In [ ]:


def plot_histogram(data, title, xlabel, ylabel, bins=50, xlim=None):
    plt.hist(data, bins=bins, alpha=0.7)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if xlim:
        plt.xlim(xlim)
    plt.grid(True)
    plt.show()

# Load original data
p_load_data = pd.read_csv('synthetic_data/p_load_data.csv', index_col=0).values
p_max_gen_data = pd.read_csv('synthetic_data/p_gen_data.csv', index_col=0).values

# Plot histograms for Load Data with normalized frequencies
plot_histogram(p_load_data.flatten(), 'Load Data', 'Load (p_mw)', 'Frequency')

# Plot histograms for RES Data with normalized frequencies
plot_histogram(p_max_gen_data.flatten(), 'RES Data', 'RES Power (p_mw)', 'Frequency')
